In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch11
!touch ch11/__init__.py

mkdir: cannot create directory ‘ch11’: File exists


In [0]:
import imp

## 11.1 Python 文化中的接口和协议

### 接口

- 接口是对象公开方法的子集，让对象在系统中扮演特定的角色
- Python 语言没有 interface 关键字，且除了抽象基类，每个类都有接口
  - 类实现或继承的公开属性，包括特殊方法，如 `__getitem__` 或 `__add__` 
- 受保护的属性或私有属性不在接口中
  - “受保护的属性” 只是采用命名约定来实现（单个前导下划线）

### 协议

- 协议与继承没有关系，一个类可能会实现多个接口，从而让实例扮演多个角色
- 协议是接口，但不是正式的（只由文档和约定定义），因此协议不能像正式接口那样施加限制
- 序列协议是 Python 最基础的协议之一

## 11.2 Python 喜欢序列

- 抽象基类的 Sequence 正式接口
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200506080552.png width=800>

###### 示例 11-3　定义 `__getitem__` 方法，只实现序列协议的一部分，这样足够访问元素、迭代和使用 `in` 运算符了

In [0]:
class Foo:
  def __getitem__(self, pos):
    return range(0, 30, 10)[pos]

In [0]:
f = Foo()
f[1]

10

In [0]:
for i in f: print(i)

0
10
20


In [0]:
20 in f

True

In [0]:
15 in f

False

- 虽然没有 `__iter__` 方法，但是 `Foo` 实例是可迭代的对象，因为发现有 `__getitem__` 方法时，Python 会调用它，传入从 `0` 开始的整数索引，尝试迭代对象（这是一种后备机制）
- 尽管没有实现 `__contains__` 方法，但是 Python 足够智能，能迭代 `Foo` 实例，因此也能使用 `in` 运算符：Python 会做全面检查，看看有没有指定的元素。

## 11.3 使用猴子补丁在运行时实现协议

###### 标准库中 `random.shuffle` 的用法

In [0]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

###### 示例 11-5　`random.shuffle` 函数不能打乱 `FrenchDeck` 实例

In [0]:
import ch1.frenchdeck
imp.reload(ch1.frenchdeck)
from ch1.frenchdeck import FrenchDeck
from random import shuffle

In [0]:
deck = FrenchDeck()
shuffle(deck)

TypeError: ignored

- 报错的原因是 “'FrenchDeck' object does not support item assignment”（`FrenchDeck` 对象不支持为元素赋值）
- `shuffle` 函数要调换集合中元素的位置，而 `FrenchDeck` 只实现了不可变的序列协议。可变的序列还必须提供 `__setitem__` 方法

###### 示例 11-6　为 `FrenchDeck` 打猴子补丁，把它变成可变的，让 `random.shuffle` 函数能处理（接续示例 11-5）

In [0]:
def set_card(deck, position, card):
  deck._cards[position] = card

In [0]:
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='J', suit='spades'),
 Card(rank='5', suit='hearts'),
 Card(rank='4', suit='spades'),
 Card(rank='2', suit='clubs'),
 Card(rank='8', suit='clubs')]

- 特殊方法 `__setitem__` 的签名在 Python 语言参考手册的 [3.3.6.Emulating containertypes](https://docs.python.org/3/reference/datamodel.html#emulatingcontainer-types)中定义
  - 语言参考中使用的参数是 `self`、`key` 和 `value`
  - 而示例中使用的是 `deck`、`position` 和 `card`
    - 这说明 Python 方法说到底都是普通函数，把第一个参数命名为 `self` 只是一种约定
    - 在控制台会话中使用那这个参数没问题，不过在 Python 源码文件中最好按照文档那样使用 `self`、`key` 和 `value`

### 猴子补丁

- 示例 11-6 中所采用的技术即叫做猴子补丁
  - 在运行中修改类或模块，而不改动源码
  - 猴子补丁很强大，但是打补丁的代码与要打补丁的程序耦合十分紧密，而且往往要处理隐藏和没有文档的部分
- 示例 11-6 强调了协议是动态的
  - `random.shuffle` 函数不关心参数的类型，只要那个对象实现了部分可变序列协议即可
  - 即便对象一开始没有所需的方法也没关系，后来再提供也行
- 目前，本章讨论的主题是“鸭子类型”：对象的类型无关紧要，只要实现了特定的协议即可

## 11.4 Alex Matrelli 的水禽

###### ["鸭子类型"](https://en.wikipedia.org/wiki/Duck_typing#History) 忽略对象真正类型，转而关注对象有没有实现所需要方法、签名和语义
  - 这基本上是指避免使用 `isinstance` 检查对象的类型

###### 白鹅类型指，只要 `cls` 是抽象基类，即 `cls` 的元类是 `abc.ABCMeta`，就可以使用 `isinstance(obj, cls)`
  - Python 的抽象基类有一个重要的实用优势
    - 可以使用 `register` 类方法在终端用户的代码中把某个类“声明”为一个抽象基类的“虚拟”子类
      - 为此，被注册的类必须满足抽象基类对方法名称和签名的要求，最重要的是要满足底层语义契约
      - 但是，开发那个类时不用了解抽象基类，更不用继承抽象基类
    - 如此可大大打破严格的强耦合
  - 有时，为了让抽象基类实别子类，甚至不需要注册

###### 抽象基类的本质是几个特殊方法

In [0]:
class Struggle:
  def __len__(self): return 23

In [0]:
from collections import abc
isinstance(Struggle(), abc.Sized)

True

- 无需注册，`abc.Sized` 也能把 `Struggle` 识别为自己的子类，只要实现了特殊方法 `__len__` 即可
  - 要使用正确的句法和语义实现，前者要求没有参数，后者要求返回一个非负整数，指明对象的长度
  - 如果不使用规定的句法和语义实现特殊方法，如 `__len__`，会导致非常严重的问题）
- 如果实现的类体现了 `numbers`、`collections.abc` 或其他框架中抽象基类的概念，要么继承相应的抽象基类（必要时），要么把类注册到相应的抽象基类中
  - 开始开发程序时，不要使用提供注册功能的库或框架，要自己动手注册
  - 如果必须检查参数的类型（这是最常见的），例如检查是不是“序列”，可以采用如下代码：
    - `isinstance(the_arg, collections.abc.Sequence)`
    - 使用 `isinstance` 和 `issubclass` 测试抽象基类更为人接受
      - 之前，这两个函数用来测试鸭子类型，但用于抽象基类会更灵活
      - 因为如果某个组件没有继承抽象基类，事后还可以注册，让显式类型检查通过
    - 即便是抽象基类，也不能滥用 `isinstance` 检查，用得多了可能表明面向对象设计得不好
      - 在一连串 `if/elif/elif` 中使用 `isinstance` 做检查，然后根据对象的类型执行不同的操作，通常是不好的做法
      - 此时应该使用多态，即采用一定的方式定义类，让解释器把调用分派给正确的方法
    - 如果必须强制执行 API 契约，通常可以使用 isinstance 检查抽象基类
      - 这对采用插入式架构的系统来说特别有用

###### 示例 11-7　使用鸭子类型处理单个字符串或由字符串组成的可迭代对象

[collections.namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) 中对 `field_names` 参数的处理方式

``` python
try: # 假设是单个字符串（EAFP 风格，即“取得原谅比获得许可容易”）
  field_names = field_names.replace(',', ' ').split()  # 把逗号替换成空格，然后拆分成名称列表
except AttributeError:  # 如果 field_names 不是字符串……没有 .replace 方法，或者返回值不能使用 .split 方法拆分
  pass
field_names = tuple(field_names)  # 为了确保的确是可迭代对象，也为了保存一份副本，使用所得值创建一个元组
```

###### 抽象基类的使用
- 抽象基类是用于封装框架引入的一般性概念和抽象的，例如“一个序列”和“一个确切的数”
- 用户基本上不需要自己编写新的抽象基类，只要正确使用现有的抽象基类，就能获得 99.9% 的好处，而不用冒着设计不当导致的巨大风险

## 11.5 定义抽象基类的子类

###### 示例 11-8　frenchdeck2.py：`FrenchDeck2`，`collections.MutableSequence` 的子类

In [0]:
%%writefile ch11/frenchdeck2.py
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(collections.MutableSequence):
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

  def __setitem__(self, position, value):  # 为了支持洗牌，只需实现 __setitem__ 方法
    self._cards[position] = value

  def __delitem__(self, position):  # 继承 MutableSequence 的类必须实现 __delitem__ 方法
    del self._cards[positon]

  def insert(self, position, value):  # 继承 MutableSequence 的类必须实现 insert 方法
    self._cards.insert(position, value)

Writing ch11/frenchdeck2.py


###### 抽象基类子类的特点

- 导入时（加载并编译 frenchdeck2.py 模块时），Python 不会检查抽象方法的实现
- 在运行时实例化 `FrenchDeck2` 类时才会真正检查
  - 因此，如果没有正确实现某个抽象方法，Python 会抛出 TypeError 异常
    - 错误消息设为"Can't instantiate abstract classFrenchDeck2 with abstract methods __delitem__, insert"
  - 因此，即便 `FrenchDeck2` 类不需要 `__delitem__` 和 `insert` 提供的行为，也要实现
    - 因为 MutableSequence 抽象基类需要它们
- `Sequence` 和 `MutableSequence` 抽象基类的方法不全是抽象的，如下图所示
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200507190235.png width=600>
  - `FrenchDeck2` 从 `Sequence` 继承了几个拿来即用的具体方法：`__contains__`、`__iter__`、`__reversed__`、`index` 和`count`
  - `FrenchDeck2` 从 `MutableSequence` 继承了`append`、`extend`、`pop`、`remove` 和 `__iadd__`
- 在 `collections.abc` 中，每个抽象基类的具体方法都是作为类的公开接口实现的，因此不用知道实例的内部接口
- 要想实现子类，可以覆盖从抽象基类中继承的方法，以更高效的方式重新实现
  - 如 `__contains__` 方法会全面扫描序列
    - 但如果定义的序列按顺序保存元素，那就可以重新定义 `__contains__` 方法
      - 使用 bisect 函数做二分查找（参见 [2.8节]()），从而提升搜索速度

## 11.6 标准库中的抽象基类

### 11.6.1 `collections.abc` 模块中的抽象基类

- 标准库中有两个名为 `abc` 的模块，一个是 `collections.abc`
- 另一个就 `abc` 其中定义的是 `abc.ABC` 类，每个抽象基类都依赖于这个类，但是不用导入，除非定义新的抽象基类

In [0]:
import collections, abc

In [0]:
collections.abc

<module 'collections.abc' from '/usr/lib/python3.6/collections/abc.py'>

In [0]:
abc

<module 'abc' from '/usr/lib/python3.6/abc.py'>

#### `collections.abc` 模块中的 16 个抽象基类

- [官方文档](https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes)
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200507193640.png width=600>

##### `Iterable`、`Container` 和 `Sized`

- 各个集合应该继承这三个抽象基类，或者至少实现兼容的协议
- `Iterable` 通过 `__iter__` 方法支持迭代，`Container` 通过 `__contains__` 方法支持 `in` 运算符，`Sized` 通过 `__len__` 方法支持 `len()` 函数。

##### `Sequence`、`Mapping` 和 `Set`

- 这三个是主要的不可变集合类型，而且各自都有可变的子类
- `MutableSequence` 的详细类图见图 11-2
- `MutableMapping` 和 `MutableSet` 的类图在第 3 章中（见图 3-1 和图 3-2）。

##### `MappingView`

- 在 Python3 中，映射方法 `.items()`、`.keys()` 和 `.values()` 返回的对象分别是 `ItemsView`、`KeysView` 和 `ValuesView` 的实例
- 前两个类还从 `Set` 类继承了丰富的接口

###### `Callable` 和 `Hashable`

- 这两个抽象基类与集合没有太大的关系
- 只是因为 `collections.abc` 是标准库中定义抽象基类的第一个模块，而它们又太重要了，因此才把它们放到 `collections.abc` 模块中
- 一般没有 `Callable` 或 `Hashable` 的子类
- 这两个抽象基类的主要作用是为内置函数 `isinstance` 提供支持，以一种安全的方式判断对象能不能调用或散列

##### `Iterator`

- 其是 `Iterable` 的子类，会在第 14 章中介绍

### 11.6.2 抽象基类的数字塔

- [numbers 包](https://docs.python.org/3/library/numbers.html) 定义的是 “数字塔” (即各个抽象基类的层次结构是线性的)
- 其中 `Number` 是位于最顶端的超类，随后是 `Complex` 子类，依次往下，最底端是 `Integral` 类
  - `Number`
  - `Complex`
  - `Real`
  - `Rational`
  - `Integral`
- 如果想检查一个数是不是整数，可以使用 `isinstance(x, numbers.Integral)`
  - 这样代码就能接受 `int`、`bool`（`int` 的子类），或者外部库使用 `numbers` 抽象基类注册的其他类型
  - 为了满足检查的需要，用户始终可以把兼容的类型注册为 `numbers.Integral` 的虚拟子类
- 如果一个值可能是浮点数类型，可以使用 `isinstance(x,numbers.Real)` 检查
  - 如此代码就能接受 `bool`、`int`、`float`、`fractions.Fraction`，或者外部库（如NumPy，它做了相应的注册）提供的非复数类型
- [`decimal.Decimal`](https://docs.python.org/3/library/decimal.html) 没有注册为 `numbers.Real` 的虚拟子类
  - 没有注册的原因是，如果程序需要 `Decimal` 的精度，要防止与其他低精度数字类型混淆，尤其是浮点数

## 11.7 定义并使用一个抽象基类

- 自定义的抽象基类 `Tombola`
  - 功能是支持用户提供随机挑选的无重复类， 类似于宾果机或彩票机
    - 宾果机和彩票机是随机从有限的集合中挑选物品的机器，选出的物品没有重复，直到选完为止
- `Tombola` 抽象基类有四个方法
  - 两个是抽象方法
    - `.load(..)`: 将元素放入容器
    - `.pick()`: 从容器中随机拿出一个元素，返回选中的元素
  - 两个是具体方法
    - `.loaded()`: 如果容器中至少有一个元素，返回 `True`
    - `.inspect()`: 返回一个有序元组，由容器中的现有元素构成，不会修改容器内容
- `Tombola` 抽象基类和其三个具体实现
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200512062129.png width=800>

##### 示例 11-9　tombola.py：Tombola 是抽象基类，有两个抽象方法和两个具体方法

In [0]:
%%writefile ch11/tombola.py
import abc


class Tombola(abc.ABC):

  @abc.abstractmethod
  def load(self, iterable):
    """从可迭代对象中添加元素"""

  @abc.abstractmethod
  def pick(self):
    """随机删除元素，然后将其返回
    
    如果实例为空，这个方法应该抛出 `LookupError`
    """

  def loaded(self):
    """如果至少有一个元素，返回 `True`， 否则返回 `False` """
    return bool(self.inspect())  # 抽象基类中的具体方法只能依赖抽象基类定义的接口

  def inspect(self):
    """返回一个有序数组，由当前元素构成"""
    items = []
    while True:
      try:
        items.append(self.pick())  # 由于不知道具体的子类如何储存元素，为了得到 inspect 的结果，可以不断调用 .pick 方法，将 Tombola 清空
      except LookupError:
        break
    self.load(items)  # 之后调用 .load(...) 把所有元素放回去
    return tuple(sorted(items))


Writing ch11/tombola.py


###### 示例解析

- 抽象方法可以有实现代码。即便实现了，子类也必须覆盖抽象方法
- 但子类中可以使用 `super()` 函数调用抽象方法，为它添加功能，而不是从头开始实现
- 抽象基类可以提供具体方法，只要依赖于接口中的其他方法即可
- `Tombola` 的 `loaded` 和 `inspect` 方法比较耗时，具体的子类可以根据其自身的数据结构做得更好
- `self.pick()` 抛出 `LookupError` 这一事实也是接口的一部分，但在 Python 中无法声明，只能在文档中进行说明

###### 示例 11-10 异常类的部分层次结构

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200512064940.png width=700>

###### 示例 11-11　不符合 Tombola 要求的子类无法蒙混过关

In [0]:
import ch11.tombola
imp.reload(ch11.tombola)
from ch11.tombola import Tombola

In [0]:
class Fake(Tombola):
  def pick(self):
    return 13

In [0]:
repr(Fake)

"<class '__main__.Fake'>"

- 尝试实例化 `Fake` 时抛出了 `TypeError`
- 错误消息十分明确：Python 认为 `Fake` 是抽象类，因为它没有实现 `load` 方法，这是 `Tombola` 抽象基类声明的抽象方法之一

In [0]:
f = Fake()

TypeError: ignored

### 11.7.1 抽象基类的句法详解

- 声明抽象基类最简单的方试是继承 `abc.ABC` 或其它抽象基类
- 在 python3.4 以前，必须在 `class` 语句中使用 `metaclass = abc.ABCMeta`
  - ```python
    class Tombola(metaclass=abc.ABCMeta):
      #...
    ```
- 在 Python2 中，必须使用 `__metaclass__` 类属性
  - ```python
    class Tombola(object):  # Python2
      __metaclass__ = abc.ABCMeta
  ```

- 除了 `@abstractmethod` 之外，`abc` 模块还定义了 `@abstractclassmethod`、`@abstractstaticmethod` 和`@abstractproperty` 三个装饰器
- 后三个装饰器从 Python 3.3起废弃了，因为装饰器可以在 `@abstractmethod` 上堆叠，这三个就显得多余了
  - 与其它方法描述符一起使用时， `abstractmethod()` 应该放在最里层
  - ```python
    class MyABC(abc.ABC):
      @classmethod
      @abc.abstractmethod
      def an_abstract_classmethod(cls, ...):
        pass
  ```

### 11.7.2 定义 `Tombola` 抽象基类的子类

###### 示例 11-12　bingo.py：`BingoCage` 是 `Tombola` 的具体子类

In [0]:
%%writefile ch11/bingo.py
import random

from tombola import Tombola


class BingoCage(Tombola):
  def __init__(self, items):
    # 。random.SystemRandom 使用os.urandom(...) 函数实现 random API
    #根据 os 模块的文档，os.urandom(...)函数生成“适合用于加密”的随机字节序列。
    self._randomizer = random.SystemRandom()
    self._items = []
    self.load(items)

  def load(self, items):
    self._items.extend(items)
    self._randomizer.shuffle(self._items)

  def pick(self):
    try:
      return self._items.pop()     
    except IndexError:
      raise LookupError('pick form empty BingoCage')

  def __call__(self):
    self.pick()

Overwriting ch11/bingo.py


- `BingoCage` 从 `Tombola` 中继承了耗时的 `loaded` 方法和笨拙的 `inspect` 方法
- 这两个方法都可以覆盖， 但我们可以偷懒，直接从抽象基类中继承不是那么理想的具体方法。

###### 示例 11-13　lotto.py：`LotteryBlower` 是 `Tombola` 的具体子类，覆盖了继承的 `inspect` 和 `loaded` 方法

In [0]:
%%writefile ch11/lotto.py
import random

from tombola import Tombola


class LotteryBlower(Tombola):
  
  def __init__(self, iterable):
    self._balls = list(iterable)  # 防止修改传入的参数

  def load(self, iterable):
    self._balls.extend(iterable)

  def pick(self):
    try:
      position = random.randrange(len(self._balls))
    except ValueError:
      raise LookupError('pick from empty LotteryBlower')
    return self._balls.pop(position)

  def loaded(self):
    return bool(self._balls)

  def inspect(self):
    return tuple(sorted(self._balls))

Overwriting ch11/lotto.py


### 11.7.3 Tombola 的虚拟子类

- 白鹅类型的一个基本特性：即便不继承，也有办法把一个类注册为抽象基类的虚拟子类
  - 这样做时，我们保证注册的类忠实地实现了抽象基类定义的接口，而 Python 会相信我们，从而不做检查
  - 如果我们说谎了，那么常规的运行时异常会把我们捕获。
- 注册虚拟子类的方式是在抽象基类上调用 `register` 方法
  - 这么做之后，注册的类会变成抽象基类的虚拟子类
  - 且 `issubclass` 和 `isinstance` 等函数都能识别，但是注册的类不会从抽象基类中继承任何方法或属性
- 虚拟子类不会继承注册的抽象基类，而且任何时候都不会检查它是否符合抽象基类的接口，即便在实例化时也不会检查
  - 为了避免运行时错误，虚拟子类要实现所需的全部方法

###### 示例 11-14　tombolist.py：`TomboList` 是 `Tombola` 的虚拟子类

- 虚拟子类的 UML 类图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200512074847.png width=400>
- 代码实现

In [0]:
%%writefile ch11/tombolist.py
from random import randrange

from tombola import Tombola

@Tombola.register  # 把 Tombolist 注册为 Tombola 的虚拟子类
class TomboList(list):

  def pick(self):
    if self:  # Tombolist 从 list 中继承 __bool__ 方法，列表不为空时返回 True
      position = randrange(len(self))
      return self.pop(position)
    else:
      raise LookupError('pop from empty TomboList')
    
  load = list.extend  # Tombolist.load 与 list.extend 一样

  def loaded(self):
    return bool(self)

  def inspect(self):
    return tuple(sorted(self))

Overwriting ch11/tombolist.py


- 注册之后，可以使用 `issubclass` 和 `isinstance` 函数判断 `TomboList` 是不是 `Tombola` 的子类：

In [0]:
from ch11.tombolist import TomboList
from ch11.tombola import Tombola

In [0]:
issubclass(TomboList, Tombola)

True

In [0]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

- 类的继承关系在一个特殊的类属性中指定 —— `__mro__`，即方法解析顺序（Method Resolution Order）
- 这个属性的作用很简单，按顺序列出类及其超类，Python 会按照这个顺序搜索方法
- `Tombolist.__mro__` 中没有 `Tombola`，因此 `Tombolist` 没有从 `Tombola` 中继承任何方法

In [0]:
repr(TomboList.__mro__)

"(<class 'ch11.tombolist.TomboList'>, <class 'list'>, <class 'object'>)"

## 11.8 Tombola 子类的测试方法

- 测试中用到的两个类属性
  - `__subclasses__()`
    - 这个方法返回类的直接子类列表，不含虚拟子类
  - `_abc_registry`
    - 只在抽象基类有这个数据属性，其值为一个 `weakSet` 对象，即抽象基类注册的虚拟子类的弱引用
- 为了测试 `Tombola` 的所有子类，脚本需要迭代 `Tombola.__subclasses__()` 和 `Tombola._abc_registry` 得到的列表，然后把这个类赋值给在 `doctest` 中使用的 `ConcreteTombola`

###### 示例 11-15　tombola_runner.py：Tombola 子类的测试运行程序

In [0]:
%%writefile ch11/tombola_runner.py
import doctest

from tombola import Tombola

# 要测试的模块
import bingo, lotto, tombolist  # 导入真实子类和虚拟子类，用于测试
TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16}{1.attempted:2} tests, {1.failed:2} failed - {2}'


def main(argv):
  verbose = '-v' in argv
  real_subclasses = Tombola.__subclasses__()  # __subclasses__() 返回的列表是内存中存在的直接子代
  virtual_subclasses = list(Tombola._abc_registry)

  for cls in real_subclasses + virtual_subclasses:
    test(cls, verbose)


def test(cls, verbose=False):

  res = doctest.testfile(
      TEST_FILE,
      globs={'ConcreteTombola': cls},
      verbose=verbose,
      optionflags=doctest.REPORT_ONLY_FIRST_FAILURE
  )
  tag = 'FAIL' if res.failed else 'OK'
  print(TEST_MSG.format(cls.__name__, res, tag))


if __name__ == "__main__":
  import sys
  main(sys.argv)

Overwriting ch11/tombola_runner.py


###### 示例 11-16　tombola_tests.rst：Tombola 子类的 doctest

In [0]:
%%writefile ch11/tombola_tests.rst
===============
Tombola tests
===============

Every concrete subclass of Tombola should pass these tests.

Create and load instance form iterable::

  >>> balls = list(range(3))
  >>> globe = ConcreteTombola(balls)
  >>> globe.loaded()
  True
  >>> globe.inspect()
  (0, 1, 2)

Pick and collect balls::
  
  >>> picks = []
  >>> picks.append(globe.pick())
  >>> picks.append(globe.pick())
  >>> picks.append(globe.pick())

Check state and results::

  >>> globe.loaded()
  False
  >>> sorted(picks) == balls
  True


Reload::

  >>> globe.load(balls)
  >>> globe.loaded()
  True
  >>> picks = [globe.pick() for i in balls]
  >>> globe.loaded()
  False


Check that `LookupError` (or a subclass) is the exception thrown when the devices is empty::

  >>> globe = ConcreteTombola([])
  >>> try:
  ...   globe.pick()
  ... except LookupError as exec:
  ...   print('OK')
  OK

Load and pick 100 balls to verify that they all come out::

  >>> balls = list(range(100))
  >>> globe = ConcreteTombola(balls)
  >>> picks = []
  >>> while globe.inspect():
  ...   picks.append(globe.pick())
  >>> len(picks) == len(balls)
  True
  >>> set(picks) == set(balls)
  True

Check that the order has changed and is not simply reversed::

  >>> picks != balls
  True
  >>> picks[::-1] != balls
  True

Note: the previous 2 tests have a *very* small chance of failing
even if the implementation is OK. The probability of the 100
balls coming out, by chance, in the order they were inspect is
1/100!, or approximately 1.07e-158. It's much easier to win the
Lotto or to become a billionaire working as a programmer.Note: 

THE END

Overwriting ch11/tombola_tests.rst


###### 运行测试

In [0]:
!python3 ch11/tombola_runner.py

BingoCage       24 tests,  0 failed - OK
LotteryBlower   24 tests,  0 failed - OK
TomboList       24 tests,  0 failed - OK


## 11.9 Python 使用 register 的方式

- 虽然可将 `register` 当做装饰器来使用，但更常见的做法是将其当做函数使用，用于注册其他地方定义的类
- 在 `collecitons.abc` 模块的源码中，采用以下方式将内置类型 `tuple`、`str`、 `range`　和 `memoryview` 注册为 `Sequence` 的虚拟子类
  - ```python
  Sequence.register(tuple)
  Sequence.register(str)
  Sequence.register(range)
  Sequence.register(memoryview)
  ```
- 这些类型在导入时注册，因为必须导入才能使用抽象基类：能访问 `MutableMapping` 才能编写 `isinstance(my_dict, MutableMapping)`

## 11.10 鹅的行为有可能像鸭子

- 即使不注册，抽象基类也能把一个类识别为虚拟子类
  - 如 `abc.Sized` 能将实现了 `__len__` 方法的类视为其子类
  - 原因是 `abc.Sized` 实现了一个特殊的类方法，名为 `__subclasshook__`

###### 示例 11-17　Sized 类的源码，摘自 [Lib/_collections_abc.py](https://hg.python.org/cpython/file/3.4/Lib/_collections_abc.py#l127)

```python
class Sized(metaclass=ABCMeta):

  __slots__ = ()

  @abstractmethod
  def __len__(self):
    return 0

  @classmethod
  def __subclasshook__(cls, C):
    if cls is Sized:
      if any("__len__" in B.__dict__ for B in C.__mro__):
        return True
    return NotImplemented
```

- `__subclasshook__` 在白鹅类型中添加了一些鸭子类型的踪迹

In [0]:
a = '1E-3'

In [0]:
int(a)

ValueError: ignored